## 最小代价爬楼梯

[力扣 746](https://leetcode-cn.com/problems/min-cost-climbing-stairs/)

![](../../imgs/746.png)

In [11]:
class Solution(object):
    def minCostClimbingStairs(self, cost):
        """
        :type cost: List[int]
        :rtype: int
        """
        n = len(cost)
        # dp[i] 代表从第 i 级楼梯向上爬到顶的最小花费, 特别地, dp[n] = 0 用于辅助计算
        dp = [0] * (n+1)
        dp[n-1] = cost[n-1]
        i = n-2
        while i>=0:
            dp[i] = cost[i] + min(dp[i+1], dp[i+2])
            i-=1
        return min(dp[0], dp[1])

## 不同路径

[力扣 62](https://leetcode.cn/problems/unique-paths/)

![](../../imgs/62.png)

In [12]:
class Solution(object):
    def uniquePaths(self, m, n):
        """
        :type m: int
        :type n: int
        :rtype: int
        """
        dp = [[0] * n for _ in range(m)]
        for i in range(m):
            dp[i][-1] = 1
        for j in range(n):
            dp[-1][j] = 1
        for i in range(m-2, -1, -1):
            for j in range(n-2, -1, -1):
                dp[i][j] = dp[i+1][j] + dp[i][j+1]
        return dp[0][0]

## 不同路径 II

[力扣 63](https://leetcode.cn/problems/unique-paths-ii/)

![](../../imgs/63.png)

In [13]:
class Solution(object):
    def uniquePathsWithObstacles(self, obstacleGrid):
        """
        :type obstacleGrid: List[List[int]]
        :rtype: int
        """
        m, n = len(obstacleGrid), len(obstacleGrid[0])
        dp = [[0] * n for _ in range(m)]

        # dp init
        for i in range(m-1, -1, -1):
            if obstacleGrid[i][-1] > 0:
                break
            dp[i][-1] = 1

        for j in range(n-1, -1, -1):
            if obstacleGrid[-1][j] > 0:
                break
            dp[-1][j] = 1

        for i in range(m-2, -1, -1):
            for j in range(n-2, -1, -1):
                # 赋值 dp[i][j]
                if obstacleGrid[i][j] > 0:
                    dp[i][j] = 0
                    continue
                dp[i][j] = dp[i+1][j] + dp[i][j+1]

        return dp[0][0]

## 整数拆分

[力扣 343](https://leetcode.cn/problems/integer-break/)

### Naive DP

![](../../imgs/343_naive_dp.png)

In [14]:
class Solution(object):
    def integerBreak(self, n):
        """
        :type n: int
        :rtype: int
        """
        # dp[i][j] 意味着: 和为 i 的 j 个数可以达到的最大成绩, 第 0 行和第 0 列闲置不用, i >= j 才有意义
        # i == 1...n
        # j == 2...n
        # dp init
        dp = [[0] * (n+1) for _ in range(n+1)]
        for i in range(1, n+1):
            dp[i][1] = i

        # dp sweep
        for j in range(2, n+1):
            for i in range(1, n+1):
                # calculate dp[i][j]
                potentials = []
                for num in range(1, i+1):
                    potentials.append(num * dp[i-num][j-1])
                dp[i][j] = max(potentials)

        return max(dp[n][2:])

### Optimized DP

因为:

如果一个最大的乘积里面包含一个大于或等于4的因数f，那么你可以用 $2$ 和 $f-2$ 来替换它，而不会影响结果的最大值。因为$$2*(f-2)=2f-4>=f$$。所以你永远不需要大于或等于4的因数，只需要用$1$、$2$和$3$这三个因数就可以了。但$1$是很浪费的，只有在$n$等于$2$或$3$时才需要用到。

![](../../imgs/343_optimized_dp.png)

In [15]:
class Solution:
    def integerBreak(self, n):
        if n <= 3:
            return n - 1

        dp = [0] * (n + 1)
        dp[2] = 1
        for i in range(3, n + 1):
            dp[i] = max(2 * (i - 2), 2 * dp[i - 2], 3 * (i - 3), 3 * dp[i - 3])

        return dp[n]

## 不同的二叉搜索树

[力扣 96](https://leetcode.cn/problems/unique-binary-search-trees/)

In [16]:
class Solution(object):
    def numTrees(self, n):
        """
        :type n: int
        :rtype: int
        """
        # dp[i] 代表有 i 个节点的二叉搜索树的个数
        # dp init(非常重要~)
        dp = [0] * (n + 1)
        dp[0] = 1

        # dp sweep
        for i in range(1, n+1):
            # 计算 dp[i], 即二叉树总共有 i 个节点，其中有 (i-1) 个孩子节点
            for j in range(i):
                # j 代表左子树的节点数目: 0...i-1
                # 对应地，右子树节点数目为: i-1-j
                dp[i] += dp[j] * dp[i-1-j]

        return dp[n]

## 分割等和子集

这道题和 0-1背包问题很类似, 都涉及在一个数组里求子序列和, 并使之满足一定条件。

[力扣 416](https://leetcode.cn/problems/partition-equal-subset-sum/)

![](../../imgs/416.png)

In [17]:
class Solution(object):
    def canPartition(self, nums):
        """
        :type nums: List[int]
        :rtype: bool
        """
        # Special case
        n = len(nums)
        if n < 2:
            return False
        total_sum = sum(nums)
        if total_sum % 2 == 1:
            return False
        nums.sort()
        target = total_sum // 2
        if nums[-1] > target:
            return False
        # dp, shape (n, target+1), dp[i][j] 从第 i 个数字往后作为选取集合, 存在一个子集的和为 j, 最终目标: dp[0][target]
        # dp init, 初始化 dp[n-1][*]
        dp = [[False] * (target+1) for _ in range(n)]
        dp[n-1][nums[n-1]] = True

        for i in range(n-2, -1, -1):
            for j in range(1, target+1):
                dp[i][j] = dp[i+1][j-nums[i]] or dp[i+1][j]

        return dp[0][target]

## 最后一块石头的重量

[力扣 1046](https://leetcode.cn/problems/last-stone-weight/)

![](../../imgs/1046.png)

In [18]:
class Solution(object):
    def lastStoneWeight(self, stones):
        """
        :type stones: List[int]
        :rtype: int
        """
        # general case
        while len(stones) > 1:
            stones.sort()
            remains = abs(stones[-1] - stones[-2])
            stones = stones[:-2]
            if remains > 0:
                stones = [remains] + stones
        return 0 if len(stones) == 0 else stones[0]


## 最后一块石头的重量 II

[力扣 1049](https://leetcode.cn/problems/last-stone-weight-ii/)

这道题也是 0-1 背包问题, 问题的关键在于, 这个问题可以约化为:

**把石头分为尽可能重量相近的两堆, 各堆总重量之差即是答案**

==>

**寻找不超过但尽可能接近总重一半的子序列**

==>

**0-1 背包问题, weight <=> value**

![](../../imgs/1049.png)

In [19]:
class Solution(object):
    def lastStoneWeightII(self, stones):
        """
        :type stones: List[int]
        :rtype: int
        """
        # 0. 初始化
        total_weight = sum(stones)
        target = total_weight // 2
        n = len(stones)

        if n == 1:
            return stones[0]

        # 0-1 背包，在总重不超过 target 的情况下，可以达到的最大价值(尽可能达到 target)
        # dp[i][j] 代表从第 i 件开始往后, 总重不超过 j 时能获得的最大价值

        # 1. dp init, 初始化 dp[i][*] 这一行, i 是从右向左,第一个满足 stones[i] < target 的石头
        dp = [0] * (target + 1)
        i = n-1
        while stones[i] > target:
            i -= 1
        for j in range(stones[i], target+1):
            dp[j] = stones[i]

        # 2. dp sweep
        i -= 1
        while i >= 0:
            new_dp = dp[:]
            for j in range(target + 1):
                # 计算 dp[i][j], 从下往上, 一行一行地 sweep
                if j >= stones[i]:
                    new_dp[j] = max(dp[j], stones[i] + dp[j-stones[i]])
            dp = new_dp
            i -= 1

        return abs(total_weight - 2*dp[target])


## 最小路径和

[力扣 64](https://leetcode-cn.com/problems/minimum-path-sum/)


### 解法一: DP init 不够优雅

我初始化了最后一行和最后一列，实际上最后一行足矣

![](../../imgs/64.png)

In [19]:
class Solution:
    def minPathSum(self, grid):
        m, n = len(grid), len(grid[0])

        # dp init, 初始化最后一行和最后一列
        dp = [[0] * n for _ in range(m)]

        dp[-1][-1] = grid[-1][-1]
        for i in range(m-2, -1, -1):
            dp[i][-1] = grid[i][-1] + dp[i+1][-1]
        for j in range(n-2, -1, -1):
            dp[-1][j] = grid[-1][j] + dp[-1][j+1]

        # dp sweep, 从下往上, 从右往左
        for i in range(m-2, -1, -1):
            for j in range(n-2, -1, -1):
                dp[i][j] = grid[i][j] + min(dp[i+1][j], dp[i][j+1])

        return dp[0][0]


### 解法二: 优化 DP init, 就地 DP


![](../../imgs/64_2.png)

In [1]:
class Solution:
    def minPathSum(self, grid):
        m, n = len(grid), len(grid[0])

        # dp sweep, 从下往上, 从右往左
        for i in range(m-1, -1, -1):
            for j in range(n-1, -1, -1):
                if i == m-1 and j == n-1:
                    pass
                elif i == m-1:
                    grid[i][j] += grid[i][j+1]
                elif j == n-1:
                    grid[i][j] += grid[i+1][j]
                else:
                    grid[i][j] += min(grid[i][j+1], grid[i+1][j])
        return grid[0][0]
